# DCGAN

[notebook](https://github.com/nttuan8/GAN_Tutorial/blob/master/G2/DCGAN.ipynb)

### References

[Bài 2: Deep Convolutional Generative Adversarial Network (DCGAN)](https://nttuan8.com/bai-2-deep-convolutional-gan-dcgan/)

[Bài 3: Conditional GAN (cGAN)](https://nttuan8.com/bai-3-conditional-gan-cgan/)

[Bài 4: Least Squares Generative Adversarial Networks (LSGAN)](https://nttuan8.com/bai-4-least-squares-generative-adversarial-networks-lsgan/)

[Bài 5: GAN evaluation](https://nttuan8.com/bai-5-gan-evaluation/)